# Pandas и большие файлы

In [1]:
import pandas as pd

### Упражнение
Для каждого пользователя user_id из файла sales_db.csv посчитайте самую дорогую покупку (в столбце cost)

In [2]:
df = pd.read_csv('sales_db.csv')
df.head()

,user_id,category,cost
0,1,Электроника,87
1,1,Дача,10
2,1,Дети,2
3,1,Дача,199
4,2,Авто,3


In [4]:
df.groupby('user_id').max()[['cost']]

,cost
user_id,
1,199
2,35
3,8
9,17


In [5]:
df

,user_id,category,cost
0,1,Электроника,87
1,1,Дача,10
2,1,Дети,2
3,1,Дача,199
4,2,Авто,3
5,2,Авто,35
6,3,Дети,8
7,9,Дети,17


In [15]:
def calc(user_data):
    return max([x[2] for x in user_data])

In [23]:
def calc_category(user_data):
    return sorted(user_data, key=lambda x: -x[2])[0][1]

In [24]:
previous_user = None
user_data = []

with open('sales_db.csv') as f:
    header = f.readline()
    
    for line in f:
        line = line.strip().split(',')
        
        user_id, category, cost = line
        cost = float(cost)
        # print(user_id, previous_user)
        
        if previous_user == None:
            previous_user = user_id
        else:
            if user_id == previous_user:
                user_data.append([user_id, category, cost])
            else:
                # вычисление
                # print('Пора делать вычисление!')
                # print(user_data)
                
                calculation = calc_category(user_data)
                print(previous_user, calculation)
                
                user_data = []
                user_data.append([user_id, category, cost])
                previous_user = user_id
        
calculation = calc_category(user_data)
print(previous_user, calculation)    

1 Дача
2 Авто
3 Дети
9 Дети


# Объединение датафреймов

### Данные со слайдов

In [25]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad']
    }
)

visits = visits[['user_id', 'source']]
visits

,user_id,source
0,11,ad
1,22,yandex
2,55,email
3,11,google
4,77,ad


In [26]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
    }
)

purchases = purchases[['user_id', 'category']]
purchases

,user_id,category
0,11,Спорт
1,22,Авто
2,55,Дача
3,11,Спорт
4,99,Авто


In [27]:
visits_grouped = visits.groupby('user_id').count()
visits_grouped.rename(columns={'source': 'visits'}, inplace=True)
visits_grouped

,visits
user_id,
11,2
22,1
55,1
77,1


In [28]:
visits.groupby('user_id').count().reset_index()

,user_id,source
0,11,2
1,22,1
2,55,1
3,77,1


In [29]:
purchases_pivot = purchases.pivot_table(index='user_id', columns='category', values='user_id', 
                                        aggfunc='size', fill_value=0)
purchases_pivot

category,Авто,Дача,Спорт
user_id,,,
11,0,0,2
22,1,0,0
55,0,1,0
99,1,0,0


In [30]:
purchases_pivot.reset_index()

category,user_id,Авто,Дача,Спорт
0,11,0,0,2
1,22,1,0,0
2,55,0,1,0
3,99,1,0,0


In [31]:
visits_grouped.join(purchases_pivot)

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### LEFT join
Каждой строчке в левой таблице ищет соответствие в правой

In [34]:
pd.join(visits_grouped, purchases_pivot, how='left')

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [32]:
visits_grouped.join(purchases_pivot, how='left')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0.0,0.0,2.0
22,1,1.0,0.0,0.0
55,1,0.0,1.0,0.0
77,1,NaN,NaN,NaN


### RIGHT join
Каждой строчке в правой таблице ищет соответствие в левой

In [35]:
visits_grouped.join(purchases_pivot, how='right')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0,0,2
22,1.0,1,0,0
55,1.0,0,1,0
99,NaN,1,0,0


### Упражнение
Дана статистика:
- ID клиентов и их имена (датафрейм clients)
- статистика доходов (earnings)
- статистика расходов (spending)

Определите имена клиентов, расходы которых превышают доходы.

In [39]:
# подсказка - по умолчанию в методе merge объединение НЕ left join

?pd.DataFrame.merge

In [36]:
clients = pd.DataFrame(
    {
        'id': [43018, 48329, 51043, 74943, 75029],
        'name': ['Марков Илья', 'Зарицкая Елизавета', 'Благова Дарья', 'Слепова Елена', 'Гордецкий Максим'],
    }
)

clients

,id,name
0,43018,Марков Илья
1,48329,Зарицкая Елизавета
2,51043,Благова Дарья
3,74943,Слепова Елена
4,75029,Гордецкий Максим


In [37]:
earnings = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'debit': [34500, 12400, 89044, 5355, 19800],
    }
)

earnings

,id,debit
0,51043,34500
1,48329,12400
2,74943,89044
3,75029,5355
4,43018,19800


In [38]:
spending = pd.DataFrame(
    {
        'id': [51043, 48329, 74943, 75029, 43018],
        'credit': [22990, 2500, 69880, 6000, 29000],
    }
)

spending

,id,credit
0,51043,22990
1,48329,2500
2,74943,69880
3,75029,6000
4,43018,29000


In [44]:
clients.merge(earnings, how='left').merge(spending)

,id,name,debit,credit
0,43018,Марков Илья,19800,29000
1,48329,Зарицкая Елизавета,12400,2500
2,51043,Благова Дарья,34500,22990
3,74943,Слепова Елена,89044,69880
4,75029,Гордецкий Максим,5355,6000


In [45]:
total = (
    clients
    .merge(earnings, how='left')  # добавляем доходы
    .merge(spending)  # добавляем расходы
)

total.loc[ total['debit'] - total['credit'] < 0 ][['name']]


,name
0,Марков Илья
4,Гордецкий Максим


In [41]:
cl_er = clients.merge(earnings, how = 'left') 
full_table = cl_er.merge(spending, how = 'left') 

full_table[full_table.debit<full_table.credit]


,id,name,debit,credit
0,43018,Марков Илья,19800,29000
4,75029,Гордецкий Максим,5355,6000


In [42]:
clients.merge(earnings,on='id').merge(spending, on='id').query('credit > debit')

,id,name,debit,credit
0,43018,Марков Илья,19800,29000
4,75029,Гордецкий Максим,5355,6000


### INNER join
Оставляет строчки, которые есть в обеих таблицах

In [46]:
visits_grouped.join(purchases_pivot, how='inner')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2,0,0,2
22,1,1,0,0
55,1,0,1,0


### Outer join
Оставляет все строчки

In [47]:
visits_grouped.join(purchases_pivot, how='outer')

,visits,Авто,Дача,Спорт
user_id,,,,
11,2.0,0.0,0.0,2.0
22,1.0,1.0,0.0,0.0
55,1.0,0.0,1.0,0.0
77,1.0,NaN,NaN,NaN
99,NaN,1.0,0.0,0.0


# Конкатенация таблиц

In [48]:
'123' + '345'

'123345'

In [49]:
[1, 2] + [3, 4]

[1, 2, 3, 4]

In [ ]:
UNION ALL

In [50]:
a = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_a': [1, 2, 3]})
b = pd.DataFrame({'date': ['2020-01-01', '2020-01-02', '2020-01-03'], 'value_b': [3, 4, 5]})

In [51]:
a

,date,value_a
0,2020-01-01,1
1,2020-01-02,2
2,2020-01-03,3


In [52]:
b

,date,value_b
0,2020-01-01,3
1,2020-01-02,4
2,2020-01-03,5


In [53]:
pd.concat([a, b])

/Users/kbashevoy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,date,value_a,value_b
0,2020-01-01,1.0,NaN
1,2020-01-02,2.0,NaN
2,2020-01-03,3.0,NaN
0,2020-01-01,NaN,3.0
1,2020-01-02,NaN,4.0
2,2020-01-03,NaN,5.0


In [54]:
# объединение по горизонтали
pd.concat([a, b], axis=1)

,date,value_a,date,value_b
0,2020-01-01,1,2020-01-01,3
1,2020-01-02,2,2020-01-02,4
2,2020-01-03,3,2020-01-03,5


In [55]:
pd.concat([a, b], axis=1)['date']

,date,date
0,2020-01-01,2020-01-01
1,2020-01-02,2020-01-02
2,2020-01-03,2020-01-03


### Дубликаты при объединении таблиц

In [56]:
ratings = pd.read_csv('ratings_example.txt', sep = '\t')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [57]:
movies = pd.read_csv('movies_example.txt', sep = '\t')
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
2,31,Dangerous Minds (1995),Drama


In [58]:
# ¯\_(ツ)_/¯

ratings.merge(movies, how='left', on='movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [59]:
joined = ratings.merge(movies, how='left', on='movieId')
len(ratings) == len(joined)

False

In [60]:
movies.drop_duplicates(subset = 'movieId', keep = 'first', inplace = True)
movies.head()

,movieId,title,genres
0,31,Dangerous Minds (1995),Drama
1,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


In [61]:
ratings.merge(movies, how = 'left', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama


In [62]:
joined = ratings.merge(movies, how='left', on='movieId')
len(ratings) == len(joined)

True

In [63]:
ratings.merge(movies, how = 'right', on = 'movieId')

,userId,movieId,rating,timestamp,title,genres
0,1.0,31,2.5,1.260759e+09,Dangerous Minds (1995),Drama
1,NaN,32,NaN,NaN,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller


### Упражнение
Объедините датафреймы с визитами и покупками на сайте по ключу date. Обратите внимание, что в датафрейме визитов имеются дубликаты по дате.

In [64]:
visits = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-01', '2019-11-02', '2019-11-02', '2019-11-03'], 
     'source': ['organic', 'paid', 'organic', 'paid', 'organic'], 
     'visits': [16825, 1952, 21890, 376, 19509]}
)

visits

,date,source,visits
0,2019-11-01,organic,16825
1,2019-11-01,paid,1952
2,2019-11-02,organic,21890
3,2019-11-02,paid,376
4,2019-11-03,organic,19509


In [65]:
orders = pd.DataFrame(
    {'date': ['2019-11-01', '2019-11-02', '2019-11-03'],
     'orders': [198, 225, 201]}
)

orders

,date,orders
0,2019-11-01,198
1,2019-11-02,225
2,2019-11-03,201


In [74]:
visits.merge(orders)  # неверно

,date,source,visits,orders
0,2019-11-01,organic,16825,198
1,2019-11-01,paid,1952,198
2,2019-11-02,organic,21890,225
3,2019-11-02,paid,376,225
4,2019-11-03,organic,19509,201


In [75]:
visits.groupby('date').sum()

,visits
date,
2019-11-01,18777
2019-11-02,22266
2019-11-03,19509


In [66]:
visits.groupby('date').sum().merge(orders, on='date')

,date,visits,orders
0,2019-11-01,18777,198
1,2019-11-02,22266,225
2,2019-11-03,19509,201


In [79]:
svod['visits'] = svod['organic'] + svod['paid']
svod = svod.merge(orders, how = 'left')
svod

,date,organic,paid,visits,orders
0,2019-11-01,16825,1952,18777,198
1,2019-11-02,21890,376,22266,225
2,2019-11-03,19509,0,19509,201


In [81]:
svod.visits.sum() == visits.visits.sum()

True

In [82]:
svod.orders.sum() == orders.orders.sum()

True

In [76]:
svod = visits.pivot_table(index= 'date', columns = 'source', values = 'visits', aggfunc = 'sum', fill_value = 0).reset_index() 
svod.merge(orders, how = 'left')


,date,organic,paid,orders
0,2019-11-01,16825,1952,198
1,2019-11-02,21890,376,225
2,2019-11-03,19509,0,201


In [69]:
visits_extended = pd.DataFrame({'date': visits['date'].unique()}).set_index('date') 

for x in visits['source'].unique(): 
    visits_extended['visits_'+x] = visits[visits.source == x][['date','visits']].set_index('date') 
    visits_extended = visits_extended.fillna(0) 
    visits_extended.reset_index().merge(orders, on='date')
    
visits_extended
    

,visits_organic,visits_paid
date,,
2019-11-01,16825,1952.0
2019-11-02,21890,376.0
2019-11-03,19509,0.0


In [71]:
joined = visits.merge(orders, how ='left').groupby('date').max()
joined

,source,visits,orders
date,,,
2019-11-01,paid,16825,198
2019-11-02,paid,21890,225
2019-11-03,organic,19509,201


In [72]:
(visits.pivot_table(index='date', columns='source', values='visits')
 .reset_index()
 .merge(orders, how='left', on='date')
)


,date,organic,paid,orders
0,2019-11-01,16825.0,1952.0,198
1,2019-11-02,21890.0,376.0,225
2,2019-11-03,19509.0,NaN,201


### Оптимизация хранения данных

In [83]:
# 2.4mb
ratings = pd.read_csv('ml-latest-small/ratings.csv')

# 0.5mb
movies = pd.read_csv('ml-latest-small/movies.csv')
joined = ratings.merge(movies, how='left', on='movieId')

In [ ]:
joined.head()

In [84]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [88]:
len(ratings)

100004

In [85]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [87]:
len(movies)

9125

In [ ]:
len(ratings) == len(joined)

In [86]:
joined.to_csv('joined_ratings.csv', index=False)

In [ ]:
joined.head()

In [ ]:
logs = joined[['userId', 'movieId', 'rating']].head()

In [ ]:
len(joined[['movieId', 'title', 'genres']].drop_duplicates())

### Какой жанр имеет самые высокие рейтинги?

In [89]:
import numpy as np

In [90]:
genres = ['Drama', 'Action', 'Thriller']

In [91]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [92]:
len(ratings)

100004

In [93]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [94]:
len(movies)

9125

In [95]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [96]:
# рекомендуемая проверка на возможные дубликаты

len(ratings) == len(joined)

True

### Считаем рейтинг жанров

In [ ]:
# еще раз список жанров

genres = ['Drama', 'Action', 'Thriller']

In [ ]:
def genres_ratings(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    return pd.Series([row['rating'] if genre in row['genres'] else np.NaN for genre in genres])

In [ ]:
%%time
joined[genres] = joined.apply(genres_ratings, axis=1)

In [97]:
def genres_ratings_version_2(row):
    """Возвращает рейтинг, если он есть в списке жанров данного фильма"""
    
    for genre in genres:
        if genre in row.genres:
            row[genre] = row.rating
    
    return rating

In [98]:
joined[genres] = joined.apply(genres_ratings, axis=1)
joined.head()

NameError: name 'genres_ratings' is not defined

### Упражнение
Выведите средний рейтинг каждого жанра из списка genres

### К домашнему заданию, задача 2
Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

In [ ]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

In [ ]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

In [ ]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base